In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
from collections import deque, defaultdict
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm
import functools, itertools

In [2]:
input_text = au.read_txt_file_lines()
designs = input_text[0].split(', ')
patterns = input_text[2:]

In [14]:
## This algorithm works, but is too slow:
possible = 0 

for p_start in patterns:

    queue = deque([p_start])
    
    while len(queue) > 0:
        p = queue.popleft()

        for d in designs:
            if p[:len(d)] == d:
                if len(p) == len(d):
                    possible += 1
                    queue = []
                    break 

                queue.append(p[len(d):])

In [22]:
## inspecting data: all single letters are available except u(!!)

dict_designs = defaultdict(list)
for d in designs:
    dict_designs[len(d)].append(d)

print(dict_designs[1])
print(dict_designs[2])

['r', 'g', 'w', 'b']
['ug', 'wb', 'bw', 'rr', 'br', 'ru', 'gb', 'rg', 'gw', 'bu', 'rb', 'wu', 'ww', 'wg', 'ur', 'uw', 'bg', 'ub', 'rw', 'gr', 'gg', 'bb', 'wr']


Can we alter the algorithm to tailor towards `u`?
- We only need to find fancy patterns to match the `u`'s
- Idea: find the first `u`, match it to all designs (with `u`), if successful then just need consider what comes after.
- Runtime: 2mins. Note: could be (a lot?) faster by grouping designs by len and if 1 works skipping all others.

In [30]:
tmp = [d for d in designs if 'u' in d]
designs_u = []
for d in tmp:
    for i_s, s in enumerate(d):
        if s == 'u':  # first occurence
            designs_u.append((d, i_s))
            break

possible = 0

for p_start in tqdm(patterns):
    if 'u' not in p_start:
        possible += 1
        continue

    queue = deque([p_start])
    
    while len(queue) > 0:
        p = queue.popleft()
        if 'u' not in p:
            possible += 1
            queue = []
            break 

        for i_s, s in enumerate(p):  # find ind of first u
            if s == 'u':  # first occurrence:
                break 

        for (d, i_u) in designs_u:
            if i_u > i_s:  # d u occurs later than p u 
                continue 

            if p[i_s - i_u:i_s - i_u + len(d)] == d:
                if 'u' not in p[i_s - i_u + len(d):]:
                    possible += 1
                    queue = []
                    break 
                queue.append(p[i_s - i_u + len(d):])



100%|██████████| 400/400 [01:55<00:00,  3.46it/s]


In [31]:
possible

333

## part 2 
- essentially just remove break clause of original algorithm.. But would take ages to find all perms. 
- Tried implementing a lazy, cached approach but not fast enough .. Started again with a forward pass and that worked straight away! see below

In [22]:
input_text = au.read_txt_file_lines()
designs = input_text[0].split(', ')
patterns = input_text[2:]
max_len_d = max([len(x) for x in designs])
print('max len design', max_len_d, '. n designs:', len(designs))


max len design 8 . n designs: 447


In [ ]:
print(designs)
dict_designs_by_len_exact = {x: [d for d in designs if len(d) == x] for x in range(1, max_len_d + 1)}
set_designs = set(designs)

total = 0
for p in tqdm(patterns):
    list_counts = [0] * len(p) 
    for ind, char in enumerate(p):  # iteratively build up number of combinations possible
        n_chars = ind + 1

        if n_chars <= max_len_d:  # if it could be a design, check
            if p[:n_chars] in dict_designs_by_len_exact[n_chars]:
                list_counts[ind] = 1

        if n_chars == 1:
            continue 

        for size_d in range(1, min(max_len_d + 1, n_chars)):  # go back through possible inds that designs (of various sizes) could reach
            candidate_d = p[ind - size_d + 1:ind + 1]
            if candidate_d in set_designs:  # if this design exists, then we get new permutations (old count * 1)
                list_counts[ind] += list_counts[ind - size_d]

    total += list_counts[-1]

total





['ggbwrgwu', 'bbbgru', 'wwbg', 'gbuuruwu', 'bwww', 'r', 'guw', 'gwgg', 'ggbuwwr', 'wwb', 'wrwggwr', 'wwrbuwu', 'bgrr', 'urugr', 'bww', 'g', 'wurb', 'gwug', 'ruwg', 'gubw', 'bgbbwr', 'gurubr', 'www', 'uwww', 'brug', 'bwgr', 'bbgu', 'gbguuww', 'bgu', 'bgrbbrg', 'bbu', 'bwuwg', 'bbg', 'bgr', 'rgu', 'rwbbb', 'bwrw', 'gwg', 'buuwg', 'bbbwuu', 'bbwgb', 'ggwwr', 'wrww', 'wgr', 'bwb', 'ugr', 'gbwgwgbu', 'wggug', 'wuu', 'rbuw', 'rggww', 'gwrguu', 'gru', 'gubu', 'wbgbw', 'uwru', 'gggrr', 'gwwuw', 'wbbwrr', 'rgg', 'wbwgubrr', 'ruwuug', 'bbuw', 'gwbwgwgr', 'gguwrugr', 'urrbgg', 'ubwr', 'wwu', 'gbw', 'ggwbu', 'rug', 'bgb', 'ruur', 'bwg', 'wbgrrbu', 'wrrguww', 'rrbw', 'brb', 'gbg', 'rrrugrb', 'rrr', 'rwr', 'rbg', 'grwurw', 'wurr', 'gguu', 'ubuug', 'bgguuwwr', 'bwu', 'wwr', 'ggr', 'brgg', 'bugbrgu', 'ggu', 'w', 'bbubrw', 'ug', 'uwrru', 'wbwgrbu', 'wb', 'wgw', 'ruwbrg', 'bw', 'burgrurb', 'bwgb', 'wgg', 'bwrbbrww', 'rbu', 'wgb', 'ubu', 'brwgggbb', 'wbgbg', 'brrgb', 'rwb', 'bbbggwb', 'uwgggw', 'rr', 'br

100%|██████████| 400/400 [00:00<00:00, 13296.89it/s]


678536865274732

## Old recursion code that didn't work (fast enough):

In [ ]:
assert False
max_len_d = max([len(x) for x in designs])
print('max len design', max_len_d, '. n designs:', len(designs))
# dict_designs_by_len
dict_designs_by_len = {x: [d for d in designs if len(d) <= x] for x in range(1, max_len_d + 1)}
dict_designs_by_len_exact = {x: [d for d in designs if len(d) == x] for x in range(1, max_len_d + 1)}

@functools.cache  # cannot cache the yield function .. 
def get_perms(s):
    return list(_get_perms(s))

def _get_perms(s):
    yielded = False 
    if len(s) < max_len_d + 1 and s in dict_designs_by_len_exact[len(s)]:
        # yielded = True
        yield [len(s)]

    for ind in range(len(s) - 1, 0, -1):
        for x in get_perms(s[:ind]):
            for y in get_perms(s[ind:]):
                # yielded = True
                yield x + y

    # if not yielded:
    #     yield []

def get_n_perms(s):
    total = 0
    set_p = set()
    for perm in get_perms(s):
        # print('r', perm)
        if tuple(perm) not in set_p:
            set_p.add(tuple(perm))
            total += 1
    return total

print(designs)
# print(get_n_perms('brb'))

total_all = 0
for p in tqdm(patterns[:2]):
    t = get_n_perms(p[:15])
    print(p, t)
    total_all += t 


max len design 8 . n designs: 447
['ggbwrgwu', 'bbbgru', 'wwbg', 'gbuuruwu', 'bwww', 'r', 'guw', 'gwgg', 'ggbuwwr', 'wwb', 'wrwggwr', 'wwrbuwu', 'bgrr', 'urugr', 'bww', 'g', 'wurb', 'gwug', 'ruwg', 'gubw', 'bgbbwr', 'gurubr', 'www', 'uwww', 'brug', 'bwgr', 'bbgu', 'gbguuww', 'bgu', 'bgrbbrg', 'bbu', 'bwuwg', 'bbg', 'bgr', 'rgu', 'rwbbb', 'bwrw', 'gwg', 'buuwg', 'bbbwuu', 'bbwgb', 'ggwwr', 'wrww', 'wgr', 'bwb', 'ugr', 'gbwgwgbu', 'wggug', 'wuu', 'rbuw', 'rggww', 'gwrguu', 'gru', 'gubu', 'wbgbw', 'uwru', 'gggrr', 'gwwuw', 'wbbwrr', 'rgg', 'wbwgubrr', 'ruwuug', 'bbuw', 'gwbwgwgr', 'gguwrugr', 'urrbgg', 'ubwr', 'wwu', 'gbw', 'ggwbu', 'rug', 'bgb', 'ruur', 'bwg', 'wbgrrbu', 'wrrguww', 'rrbw', 'brb', 'gbg', 'rrrugrb', 'rrr', 'rwr', 'rbg', 'grwurw', 'wurr', 'gguu', 'ubuug', 'bgguuwwr', 'bwu', 'wwr', 'ggr', 'brgg', 'bugbrgu', 'ggu', 'w', 'bbubrw', 'ug', 'uwrru', 'wbwgrbu', 'wb', 'wgw', 'ruwbrg', 'bw', 'burgrurb', 'bwgb', 'wgg', 'bwrbbrww', 'rbu', 'wgb', 'ubu', 'brwgggbb', 'wbgbg', 'brrgb', 'rw

  0%|          | 0/2 [00:00<?, ?it/s]

uububurrububbrwruubububugbgwugrbrrrbbbwwbgwrbwgb 0
